In [11]:
from lxml import etree
import pandas as pd
import re
def clean(s):
    """
    remove digits and whitespace from section titles
    """
    s = re.sub(r'[-+]?\d*\.\d+|\d+', '', s)
    return s.strip()

def relevant_text(el):
    """
    probably need some logging - general document parse exception with doc id, element id etc.
    """
    def mk(e):return "" if e is None else e
    def _cond_(t):  return mk(t.text) + " "+ mk(t.tail) if t.tag in ["text"] else "##{}##".format(t.xpath("@xml:id"))
    txt =  (mk(el.text) + " ".join([_cond_(t) for t in el.getchildren() ]) +mk(el.tail)).replace("\n", " ")
    return txt.strip()

In [12]:
tree = etree.parse("out.xml")

### Get Title meta data and Math blobs by id

In [13]:
sections = pd.DataFrame([{ #"text": n.text, 
               "data": clean(' '.join(n.itertext())), 
               "id" : n.getparent().xpath("@xml:id")
              } for n in tree.xpath(".//title")])

#later we can look at the XML blobs and or a way to use runtime parser to rexcover it rather than store it
math = pd.DataFrame([{ #"text": n.text, 
               "data": n.xpath("@tex"), 
               "id" : n.xpath("@xml:id")
              } for n in tree.xpath(".//Math")])
math

,data,id
0,[{}^{1}],[m1]
1,[{}^{2}],[m2]
2,[{}^{3}],[m3]
3,[{}^{1}],[m4]
4,[{}^{3}],[m5]
5,[{}^{3}],[m6]
6,[u_{tt}=\frac{1}{3}(-u_{xxxx}+4u_{x}^{2}+4uu_{...,[S1.p1.m1]
7,"[(u,v)]",[S1.p1.m2]
8,[\begin{split}\displaystyle u_{{t_{{2}}}}&\dis...,[S1.E1.m1]
9,[{\cal M}_{3}],[S1.p1.m3]


### parse out the text with section and math referances

In [197]:
#ns = {"n": "http://dlmf.nist.gov/LaTeXML"}

pd.DataFrame([{ #"text": n.text, 
                "text" : relevant_text(n),
               "path": tree.getpath(n), 
               "xid" : n.getparent().xpath("@xml:id")
              } for n in tree.xpath(".//p")])

,path,text,xid
0,/document/titlepage/p[1],Ref. SISSA 123/98/FM,[]
1,/document/titlepage/p[2],Reduction of bihamiltonian systems and separat...,[]
2,/document/titlepage/p[3],G. Falqui,[]
3,/document/titlepage/p[4],##['m4']##,[]
4,/document/titlepage/p[5],E–mail: falqui@sissa.it,[]
5,/document/titlepage/p[6],##['m5']##,[]
6,/document/titlepage/p[7],"Via C. Saldini 50, I-20133 Milano, Italy",[]
7,/document/titlepage/p[8],E–mail: magri@vmimat.mat.unimi.it,[]
8,/document/titlepage/p[9],##['m6']##,[]
9,/document/titlepage/p[10],"Piazzale Europa 1, I-34127 Trieste, Italy",[]


In [9]:
import re
did = 'cond-mat9906254'
num_part =re.findall(r'\d+', did)[0]
did.replace(num_part, "/"+num_part)

import os
os.path.join("~/data/",did)

'~/data/cond-mat9906254'

## arxive doc metadata

In [3]:
#'blob is failry compact so just dump it but created a types.arxiv article metadata'
#todo - find out where the categories are detailed
from urllib.request import urlopen
url = """ http://export.arxiv.org/api/query?id_list=cond-mat/9906254"""
data = urlopen(url).read()
with open("metasample.xml", "wb") as r:  r.write(data)
data

b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <link href="http://arxiv.org/api/query?search_query%3D%26id_list%3Dcond-mat%2F9906254%26start%3D0%26max_results%3D10" rel="self" type="application/atom+xml"/>\n  <title type="html">ArXiv Query: search_query=&amp;id_list=cond-mat/9906254&amp;start=0&amp;max_results=10</title>\n  <id>http://arxiv.org/api/xTnjGxQGbwi7MhliimgfuR8CJQQ</id>\n  <updated>2019-02-23T00:00:00-05:00</updated>\n  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:totalResults>\n  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>\n  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">10</opensearch:itemsPerPage>\n  <entry>\n    <id>http://arxiv.org/abs/cond-mat/9906254v1</id>\n    <updated>1999-06-16T16:27:43Z</updated>\n    <published>1999-06-16T16:27:43Z</published>\n    <title>Percolation of Supercon